# Poster Download

## Enviroment Setup

In [ ]:
%pip install pandas numpy requests Pillow dotenv

In [2]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import os
import re
from typing import Optional
import time
from dotenv import load_dotenv

## Load Processed Dataset

In [3]:
train = pd.read_csv('data/train_complete.csv')
test = pd.read_csv('data/test_complete.csv')

print(train.head())
print(test.head())

   movieId                             title          genres   imdbId  tmdbId  \
0      619                         Ed (1996)          Comedy   116165   32308   
1    33826                Saint Ralph (2004)    Comedy|Drama   384488   25248   
2     1298       Pink Floyd: The Wall (1982)   Drama|Musical    84503   12104   
3   140289              Men & Chicken (2015)    Comedy|Drama  3877674  296313   
4     3064  Poison Ivy: New Seduction (1997)  Drama|Thriller   119908   18222   

   year                                         poster_url  
0  1996  https://image.tmdb.org/t/p/w500/oSNX3zyTpRMN0E...  
1  2004  https://image.tmdb.org/t/p/w500/3MdWgqUunIBWnc...  
2  1982  https://image.tmdb.org/t/p/w500/aElHyIdF5jmctF...  
3  2015  https://image.tmdb.org/t/p/w500/6lZ1EjOoWJiZaG...  
4  1997  https://image.tmdb.org/t/p/w500/l9Z43sz0qjTrmD...  
   movieId                                   title  \
0    45635       Notorious Bettie Page, The (2005)   
1     1373  Star Trek V: The Final Fron

## Fetch Poster Images

In [4]:
movies = pd.concat([train, test], axis=0, ignore_index=True)

failed: list = []

for idx, row in movies.iterrows():
    save_path: str = f"posters/{row['movieId']}.jpg"

    try:
        time.sleep(0.02)
        response = requests.get(row['poster_url'], timeout=10)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img.save(save_path)
        else:
            failed.append(row.to_dict())
    except:
        failed.append(row.to_dict())

print(f"Total images downloaded: {len(movies) - len(failed)}")

if failed:
    failed_df = pd.DataFrame(failed)
    failed_df.to_csv('data/failed_downloads.csv', index=False)
    print(f"Failed to download {len(failed)} images. Saved to 'data/failed_downloads.csv'.")

Total images downloaded: 9334
Failed to download 3 images. Saved to 'data/failed_downloads.csv'.


### Retrying failed downloads

In [3]:
missing = pd.read_csv('data/failed_downloads.csv')

for idx, row in missing.iterrows():
    save_path: str = f"posters/{row['movieId']}.jpg"

    try:
        time.sleep(0.02)
        response = requests.get(row['poster_url'], timeout=10)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img.save(save_path)
        else:
            print(f"Failed again for movieId {row['movieId']}")
    except Exception as e:
        print(f"Exception for movieId {row['movieId']}: {e}")

All poster image downloads succeeded.